In [21]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']
play_file_infixes = ['othello']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)

In [22]:
print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
#print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
# print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
# print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


What's the matter, lieutenant?
What's the matter, lieutenant?



In [23]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['ori' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['mod' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)

# idx_col = [str(x) for x in range(len(raw))]
# idx_col = np.array(idx_col).reshape(-1,1)
# print(idx_col.shape)
print(raw.shape)
#raw = np.hstack((idx_col, raw))
raw.shape
raw = np.random.permutation(raw)

(3710, 2)


In [24]:
raw[:10]

array([["You're stripped of your power and your command, and Cassio will govern Cyprus.",
        'mod'],
       ['No one passing by?', 'mod'],
       ['Therefore, good Emilia, Give me my nightly wearing, and adieu.',
        'ori'],
       ['What didst not like?', 'ori'],
       ['Come, go with me apart.', 'ori'],
       ['The voice of Cassio.', 'ori'],
       ["She's so good at sewing, and a wonderful musician.", 'mod'],
       ['Do you perceive in all this noble company Where most you owe obedience?',
        'ori'],
       ['With any strong or vehement importunity.', 'ori'],
       ['The sooner, sweet, for you.', 'ori']], dtype='<U497')

In [25]:
from sklearn.feature_extraction import DictVectorizer
import spacy
import scipy.sparse
nlp = spacy.load('en_core_web_sm')

vectorizer = DictVectorizer()
features = []

# feature engineering
    # construct index column
# idx_col = np.array([raw[:,0]])
# idx_col = idx_col.astype('f', copy=True)

#bleu: 0-1, 1 is perfect score
#bleu = sentence_bleu([sen2],sen1)

raw_text_col = 0
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    feature_dict = {}
    pos_dict = {'NOUN_count': 0, 'ADV_count': 0, 'VERB_count': 0, 
                'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'bleu': 0}
    txt = str(raw[row_idx][raw_text_col])
    
    feature_dict['sent_len'] = len(txt)
    try:
        doc = nlp(txt)
    except IndexError as e:
        print(e)
        print("offending message [{}], len: {}, row idx: {}".format(txt, len(txt), row_idx))

    for token in doc:
        pos = token.pos_
        pos_dict[pos+'_count'] = pos_dict.get(pos+'_count', 0) + 1

    if pos_dict['NOUN_count'] == 0 or pos_dict['ADJ_count'] == 0:
        pos_dict['adj_noun_ratio'] = 0
    else:
        pos_dict['adj_noun_ratio'] = pos_dict['NOUN_count'] / pos_dict['ADJ_count'] 

    if pos_dict['VERB_count'] == 0 or pos_dict['ADV_count'] == 0:
        pos_dict['adv_verb_ratio'] = 0
    else:
        pos_dict['adv_verb_ratio'] = pos_dict['VERB_count'] / pos_dict['ADV_count'] 
        
    feature_dict.update(pos_dict)
    
    if row_idx % 1000 == 0:
        print("{} gold: {} {}\n".format(txt, raw[row_idx][LABEL_COL], feature_dict) )
        
    features.append(feature_dict)

#labels_col = np.array([raw[:,LABEL_COL]])

X = vectorizer.fit_transform(features)
print(X.shape)

#X = scipy.sparse.hstack(( idx_col.T, X, labels_col.T ))
X = X.todense()


You're stripped of your power and your command, and Cassio will govern Cyprus. gold: mod {'sent_len': 78, 'NOUN_count': 2, 'ADV_count': 0, 'VERB_count': 4, 'ADJ_count': 2, 'adv_verb_ratio': 0, 'adj_noun_ratio': 1.0, 'bleu': 0, 'PRON_count': 1, 'ADP_count': 1, 'CCONJ_count': 2, 'PUNCT_count': 2, 'PROPN_count': 2}

list index out of range
offending message [], len: 0, row idx: 49
Good lieutenant, is your general married? gold: mod {'sent_len': 41, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 1, 'ADJ_count': 4, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0.25, 'bleu': 0, 'PUNCT_count': 2}

list index out of range
offending message [], len: 0, row idx: 1053
Greetings, good general! gold: mod {'sent_len': 24, 'NOUN_count': 2, 'ADV_count': 0, 'VERB_count': 0, 'ADJ_count': 1, 'adv_verb_ratio': 0, 'adj_noun_ratio': 2.0, 'bleu': 0, 'PUNCT_count': 2}

Where is that viper? gold: ori {'sent_len': 20, 'NOUN_count': 1, 'ADV_count': 1, 'VERB_count': 1, 'ADJ_count': 0, 'adv_verb_ratio': 1.0, 'adj_noun_ra

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import time
import math
###############
# training ####
###############
# test breakout: 3500, 210
train_val_split_delim = math.floor(len(X) * .8)
val_test_split_delim = math.floor(len(X) * .9)
print("total rows: {} train val split delim: {} val test split delim: {}".format(len(X), train_val_split_delim, val_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim


X_train = X[0:train_val_split_delim]
Y_train = raw[0:train_val_split_delim,LABEL_COL]

X_val = X[train_val_split_delim:val_test_split_delim]
Y_val = raw[train_val_split_delim:val_test_split_delim,LABEL_COL]

X_test = X[val_test_split_delim:]
Y_test = raw[val_test_split_delim:,LABEL_COL]


#mdl = LogisticRegression(random_state=123)
mdl = MultinomialNB()
mdl.fit(X_train, Y_train)

np.set_printoptions(suppress=True)


total rows: 3710 train val split delim: 2968 val test split delim: 3339


In [29]:
mdl.classes_

array(['mod', 'ori'], dtype='<U3')

In [35]:
# evaluation ###
################
#second class is original

from sklearn.metrics import brier_score_loss

y_true = raw[train_val_split_delim:val_test_split_delim,LABEL_COL]


#binary evaluation
# preds = mdl.predict(X_test)
# 
# test_indices = X[val_test_split_delim:,0]
# if len(preds) != len(val_indices):
#     raise ValueError("unmatched arrays")
    
# results = []
# for (idx, pred) in zip(val_indices,preds):
#     gold_label = int(raw[int(idx)][-1])
#     pred_label = int(pred)
#     #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
#     if gold_label == pred_label:
#           results.append(1)
#     else:
#         results.append(0)
# results[:5]

preds = mdl.predict(X_test)
#y_true = [int(raw[int(idx)][-1]) for idx in val_indices]

# brier = brier_score_loss(y_true, preds[:,0]) 
# print("brier score:", brier)

#currently slightly worse than randomly permuting preds

def accuracy(preds, y_true):
    if len(preds) != len(y_true):
        raise ArgumentError("mismatched arrays!")
    num_correct = 0
    for (x,y) in zip(preds, y_true):
        if x == y:
            num_correct = num_correct + 1
    return num_correct/len(preds)

print("Accuracy: {}".format(accuracy(preds,y_true)))

Accuracy: 0.5067385444743935


In [37]:
# error analysis ##
##################


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

conf_mat = confusion_matrix(y_true, preds)
fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(conf_mat, annot=True, fmt='d',
#             xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

<Figure size 1000x1000 with 1 Axes>

In [ ]:
# test_len = len(preds)
# accuracy = sum(preds)/float(test_len)
# print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228

#pos, no bleu, 2 plays
# Mult NB: .519